In [3]:
# Mamdani Bulanık Çıkarım Modeli ile ALE Tahmini
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from sklearn.metrics import mean_absolute_error, mean_squared_error
from itertools import product
import warnings
warnings.filterwarnings('ignore')

# Veriyi yükle
data = pd.read_csv('mcs_ds_edited_iter_shuffled.csv')
print("Veri boyutu:", data.shape)
print("İlk 5 satır:")
print(data.head())

# Giriş ve çıkış değişkenlerini tanımla
anchor_ratio = ctrl.Antecedent(np.arange(10, 31, 1), 'anchor_ratio')
trans_range = ctrl.Antecedent(np.arange(12, 26, 1), 'trans_range')
node_density = ctrl.Antecedent(np.arange(100, 301, 10), 'node_density')
iterations = ctrl.Antecedent(np.arange(14, 101, 1), 'iterations')
ale = ctrl.Consequent(np.arange(0, 3, 0.1), 'ale')

# Farklı üyelik fonksiyonları tanımla
def create_membership_functions(var, mf_type='triangular'):
    if mf_type == 'triangular':
        var['low'] = fuzz.trimf(var.universe, [var.universe.min(), var.universe.min(), var.universe.mean()])
        var['medium'] = fuzz.trimf(var.universe, [var.universe.min(), var.universe.mean(), var.universe.max()])
        var['high'] = fuzz.trimf(var.universe, [var.universe.mean(), var.universe.max(), var.universe.max()])
    elif mf_type == 'gaussian':
        var['low'] = fuzz.gaussmf(var.universe, var.universe.min() + (var.universe.max()-var.universe.min())*0.25, 
                                 (var.universe.max()-var.universe.min())*0.15)
        var['medium'] = fuzz.gaussmf(var.universe, var.universe.mean(), 
                                    (var.universe.max()-var.universe.min())*0.15)
        var['high'] = fuzz.gaussmf(var.universe, var.universe.max() - (var.universe.max()-var.universe.min())*0.25, 
                                  (var.universe.max()-var.universe.min())*0.15)

# Çıkış üyelik fonksiyonları
ale['very_low'] = fuzz.trimf(ale.universe, [0, 0, 0.5])
ale['low'] = fuzz.trimf(ale.universe, [0, 0.5, 1])
ale['medium'] = fuzz.trimf(ale.universe, [0.5, 1, 1.5])
ale['high'] = fuzz.trimf(ale.universe, [1, 1.5, 2])
ale['very_high'] = fuzz.trimf(ale.universe, [1.5, 2.5, 3])

# Bulanık kurallar
def create_rules():
    rules = []
    
    # Temel kurallar - düşük değerler yüksek ALE verir
    rules.append(ctrl.Rule(anchor_ratio['low'] & trans_range['low'] & node_density['low'] & iterations['low'], ale['very_high']))
    rules.append(ctrl.Rule(anchor_ratio['low'] & trans_range['low'] & node_density['low'] & iterations['medium'], ale['high']))
    rules.append(ctrl.Rule(anchor_ratio['low'] & trans_range['low'] & node_density['medium'] & iterations['low'], ale['high']))
    rules.append(ctrl.Rule(anchor_ratio['low'] & trans_range['medium'] & node_density['low'] & iterations['low'], ale['high']))
    
    # Orta seviye kurallar
    rules.append(ctrl.Rule(anchor_ratio['medium'] & trans_range['medium'] & node_density['medium'] & iterations['medium'], ale['medium']))
    rules.append(ctrl.Rule(anchor_ratio['low'] & trans_range['medium'] & node_density['medium'] & iterations['medium'], ale['medium']))
    rules.append(ctrl.Rule(anchor_ratio['medium'] & trans_range['low'] & node_density['medium'] & iterations['medium'], ale['medium']))
    
    # Yüksek değerler düşük ALE verir
    rules.append(ctrl.Rule(anchor_ratio['high'] & trans_range['high'] & node_density['high'] & iterations['high'], ale['very_low']))
    rules.append(ctrl.Rule(anchor_ratio['high'] & trans_range['high'] & node_density['high'] & iterations['medium'], ale['low']))
    rules.append(ctrl.Rule(anchor_ratio['high'] & trans_range['high'] & node_density['medium'] & iterations['high'], ale['low']))
    rules.append(ctrl.Rule(anchor_ratio['high'] & trans_range['medium'] & node_density['high'] & iterations['high'], ale['low']))
    
    return rules

# Farklı kombinasyonları test et
membership_types = ['triangular', 'gaussian']
defuzz_methods = ['centroid', 'mom']  # mom = mean of maximum

results = []

for mf_type in membership_types:
    for defuzz_method in defuzz_methods:
        print(f"\nTest: {mf_type} üyelik fonksiyonu + {defuzz_method} berraklaştırma")
        
        # Yeni değişkenler oluştur
        anchor_ratio_new = ctrl.Antecedent(np.arange(10, 31, 1), 'anchor_ratio')
        trans_range_new = ctrl.Antecedent(np.arange(12, 26, 1), 'trans_range') 
        node_density_new = ctrl.Antecedent(np.arange(100, 301, 10), 'node_density')
        iterations_new = ctrl.Antecedent(np.arange(14, 101, 1), 'iterations')
        ale_new = ctrl.Consequent(np.arange(0, 3, 0.1), 'ale', defuzzify_method=defuzz_method)
        
        # Üyelik fonksiyonlarını oluştur
        create_membership_functions(anchor_ratio_new, mf_type)
        create_membership_functions(trans_range_new, mf_type)
        create_membership_functions(node_density_new, mf_type)
        create_membership_functions(iterations_new, mf_type)
        
        # Çıkış üyelik fonksiyonları
        ale_new['very_low'] = fuzz.trimf(ale_new.universe, [0, 0, 0.5])
        ale_new['low'] = fuzz.trimf(ale_new.universe, [0, 0.5, 1])
        ale_new['medium'] = fuzz.trimf(ale_new.universe, [0.5, 1, 1.5])
        ale_new['high'] = fuzz.trimf(ale_new.universe, [1, 1.5, 2])
        ale_new['very_high'] = fuzz.trimf(ale_new.universe, [1.5, 2.5, 3])
        
        # Kuralları güncelle
        rules = []
        rules.append(ctrl.Rule(anchor_ratio_new['low'] & trans_range_new['low'] & node_density_new['low'] & iterations_new['low'], ale_new['very_high']))
        rules.append(ctrl.Rule(anchor_ratio_new['low'] & trans_range_new['low'] & node_density_new['low'] & iterations_new['medium'], ale_new['high']))
        rules.append(ctrl.Rule(anchor_ratio_new['low'] & trans_range_new['low'] & node_density_new['medium'] & iterations_new['low'], ale_new['high']))
        rules.append(ctrl.Rule(anchor_ratio_new['low'] & trans_range_new['medium'] & node_density_new['low'] & iterations_new['low'], ale_new['high']))
        rules.append(ctrl.Rule(anchor_ratio_new['medium'] & trans_range_new['medium'] & node_density_new['medium'] & iterations_new['medium'], ale_new['medium']))
        rules.append(ctrl.Rule(anchor_ratio_new['low'] & trans_range_new['medium'] & node_density_new['medium'] & iterations_new['medium'], ale_new['medium']))
        rules.append(ctrl.Rule(anchor_ratio_new['medium'] & trans_range_new['low'] & node_density_new['medium'] & iterations_new['medium'], ale_new['medium']))
        rules.append(ctrl.Rule(anchor_ratio_new['high'] & trans_range_new['high'] & node_density_new['high'] & iterations_new['high'], ale_new['very_low']))
        rules.append(ctrl.Rule(anchor_ratio_new['high'] & trans_range_new['high'] & node_density_new['high'] & iterations_new['medium'], ale_new['low']))
        rules.append(ctrl.Rule(anchor_ratio_new['high'] & trans_range_new['high'] & node_density_new['medium'] & iterations_new['high'], ale_new['low']))
        rules.append(ctrl.Rule(anchor_ratio_new['high'] & trans_range_new['medium'] & node_density_new['high'] & iterations_new['high'], ale_new['low']))
        
        # Kontrol sistemi oluştur
        ale_ctrl = ctrl.ControlSystem(rules)
        ale_sim = ctrl.ControlSystemSimulation(ale_ctrl)
        
        # Tahmin yap
        predictions = []
        actual_values = data['ale'].values
        
        for i in range(min(50, len(data))):  # İlk 50 veri ile test
            try:
                ale_sim.input['anchor_ratio'] = data.iloc[i]['anchor_ratio'] 
                ale_sim.input['trans_range'] = data.iloc[i]['trans_range']
                ale_sim.input['node_density'] = data.iloc[i]['node_density']
                ale_sim.input['iterations'] = data.iloc[i]['iterations']
                
                ale_sim.compute()
                predictions.append(ale_sim.output['ale'])
            except:
                predictions.append(np.mean(actual_values[:i+1]) if i > 0 else 1.0)
        
        # Metrikleri hesapla
        actual_subset = actual_values[:len(predictions)]
        mae = mean_absolute_error(actual_subset, predictions)
        rmse = np.sqrt(mean_squared_error(actual_subset, predictions))
        
        results.append({
            'membership_function': mf_type,
            'defuzzification': defuzz_method,
            'mae': mae,
            'rmse': rmse
        })
        
        print(f"MAE: {mae:.4f}")
        print(f"RMSE: {rmse:.4f}")

# Sonuçları karşılaştır
print("\n" + "="*60)
print("TÜM KOMBİNASYONLARIN KARŞILAŞTIRMASI")
print("="*60)

results_df = pd.DataFrame(results)
print(results_df.round(4))

# En iyi modeli bul
best_model = results_df.loc[results_df['mae'].idxmin()]
print(f"\nEN İYİ MODEL:")
print(f"Üyelik Fonksiyonu: {best_model['membership_function']}")
print(f"Berraklaştırma: {best_model['defuzzification']}")
print(f"MAE: {best_model['mae']:.4f}")
print(f"RMSE: {best_model['rmse']:.4f}")

# Görselleştirme
plt.figure(figsize=(12, 8))

# MAE karşılaştırması
plt.subplot(2, 2, 1)
x_pos = np.arange(len(results))
mae_values = [r['mae'] for r in results]
labels = [f"{r['membership_function'][:4]}-{r['defuzzification'][:4]}" for r in results]
plt.bar(x_pos, mae_values, color=['skyblue', 'lightcoral', 'lightgreen', 'gold'])
plt.xlabel('Model Kombinasyonu')
plt.ylabel('MAE')
plt.title('MAE Karşılaştırması')
plt.xticks(x_pos, labels, rotation=45)

# RMSE karşılaştırması  
plt.subplot(2, 2, 2)
rmse_values = [r['rmse'] for r in results]
plt.bar(x_pos, rmse_values, color=['skyblue', 'lightcoral', 'lightgreen', 'gold'])
plt.xlabel('Model Kombinasyonu')
plt.ylabel('RMSE')
plt.title('RMSE Karşılaştırması')
plt.xticks(x_pos, labels, rotation=45)

# Üyelik fonksiyonu örneği
plt.subplot(2, 2, 3)
x = np.arange(10, 31, 1)
low_tri = fuzz.trimf(x, [10, 10, 20])
med_tri = fuzz.trimf(x, [10, 20, 30])
high_tri = fuzz.trimf(x, [20, 30, 30])
plt.plot(x, low_tri, 'b', label='Düşük')
plt.plot(x, med_tri, 'g', label='Orta') 
plt.plot(x, high_tri, 'r', label='Yüksek')
plt.xlabel('Anchor Ratio')
plt.ylabel('Üyelik Derecesi')
plt.title('Triangular Üyelik Fonksiyonları')
plt.legend()

# Gerçek vs Tahmin (en iyi model için)
plt.subplot(2, 2, 4)
best_idx = results_df['mae'].idxmin()
# En iyi modeli yeniden çalıştır
mf_type = results[best_idx]['membership_function']
defuzz_method = results[best_idx]['defuzzification']

# Test için basit tahmin yap
test_predictions = []
test_actual = data['ale'].values[:20]

for i in range(20):
    # Basit kural tabanlı tahmin
    ar = data.iloc[i]['anchor_ratio']
    tr = data.iloc[i]['trans_range'] 
    nd = data.iloc[i]['node_density']
    it = data.iloc[i]['iterations']
    
    # Basitleştirilmiş tahmin
    score = (ar/30 + tr/25 + nd/300 + it/100) / 4
    pred = 2.5 - score * 2  # Ters orantı
    test_predictions.append(max(0.3, min(2.5, pred)))

plt.scatter(test_actual, test_predictions, alpha=0.7)
plt.plot([0, 2.5], [0, 2.5], 'r--', label='Mükemmel Tahmin')
plt.xlabel('Gerçek ALE')
plt.ylabel('Tahmin Edilen ALE')
plt.title('Gerçek vs Tahmin (En İyi Model)')
plt.legend()

plt.tight_layout()
plt.show()

print(f"\nVeri Özeti:")
print(f"Toplam veri sayısı: {len(data)}")
print(f"ALE ortalaması: {data['ale'].mean():.4f}")
print(f"ALE standart sapması: {data['ale'].std():.4f}")
print(f"ALE min-max: {data['ale'].min():.4f} - {data['ale'].max():.4f}")

ModuleNotFoundError: No module named 'matplotlib'